In [1]:
# CoLab 上挂载云盘，本地的话不需要做这一步
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
FOLDERNAME = "人工智能/Lab5"
assert FOLDERNAME is not None, "[!] Enter the foldername."
%cd drive/My\ Drive
%cp -r $FOLDERNAME ../../
%cd ../../Lab5

Mounted at /content/drive
/content/drive/My Drive
/content/Lab5


In [11]:
!pip install transformers
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import os
import numpy as np
import torch                    
import torchvision
import time
import datetime
import wandb

from tqdm import tqdm
from pathlib import Path
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset, random_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, BertConfig, get_linear_schedule_with_warmup

In [5]:
data_path = "data/"
train_path = "train.txt"
test_path = "test_without_label.txt"

In [7]:
def tag2label(tag):
    if tag == "negative": return 0
    elif tag == "neutral": return 1
    else: return 2

def get_label_dict(path):  # 从 train.txt 读取作为训练集的序号
  with open(path, 'r') as f:
    lines = f.readlines()
    lines = [line.strip() for line in lines[1:]]
  dict_ = {}
  for line in lines:
    raw_data = line.split(",")
    dict_[raw_data[0]] = tag2label(raw_data[1])
  return dict_

def get_test_text(path):  # 从 test_without_labels.txt 读取作为测试集的序号
  with open(path, 'r') as f:
    lines = f.readlines()
    lines = [line.strip() for line in lines[1:]]
  return [line.split(",")[0] for line in lines]

  def get_text(path):   # 读取特定序号的文本数据
    with open(path, 'r', errors='ignore') as f:
        line = f.readline()
    return line.strip()

In [9]:
label_dict = get_label_dict(train_path)
label_dict_keys = list(label_dict.keys())

In [10]:
test_list = get_test_text(test_path)
test_data = [['id', 'text']]
for id_ in test_list:
  test_data.append([int(id_), get_text(data_path+id_+".txt")])
test_data = pd.DataFrame(test_data[1:], columns=test_data[0])   # 测试集数据

In [11]:
print(test_data)

       id                                               text
0       8  Energetic training today with our San Antonio ...
1    1576  Let your voice be heard! 18+ #endsuicide #blit...
2    2320  RT @Austin_Powers__: Shark Week would be so mu...
3    4912             #TheTruthCaster http://t.co/S8jvqpKq5h
4    3821  RT @jarpad: Hey #WBSDCC look what we're up to!...
..    ...                                                ...
506  1048        ??? #stunned #sunglasses #gafas #gafasdesol
507  1059  Seeing @nbdbnb in a few days #excited #or #ter...
508  1485  And the #dragon guarding our #heart is called ...
509  3195  RT @NWSAlbuquerque: Isolated strong to severe ...
510  2029  Cliff dwelling above Peacocks Gallop (recently...

[511 rows x 2 columns]


In [14]:
text_data = [['id', 'label', 'text']]
for id_ in label_dict_keys:
    try:
        text_data.append([
               int(id_), 
               int(label_dict[id_]), 
               get_text(data_path + id_ + ".txt"), 
              ])
    except: continue
text_data = pd.DataFrame(text_data[1:], columns=text_data[0])       # 训练集数据

In [15]:
text_data.describe

<bound method NDFrame.describe of         id  label                                               text
0     4597      0  RT @AmitSwami77: The conspirators have an evil...
1       26      1  Waxwing trills, Chickadees calling "here sweet...
2     4383      0  @NYSE is looking a little despondent today...?...
3      212      2  FERVENT | S,M,L | 140k free PLASTIC CLIP, keyc...
4     2626      2  Nice day chilling in the park yesterday reliev...
...    ...    ...                                                ...
3995  3944      0  RT @IraqiSecurity: Car bomb in Aden Sq, Kadhim...
3996   945      2  #Remember no one can make you #feel #inferior ...
3997   695      2  RT @RapFavorites: Ye turned Valentine's Day in...
3998  4874      0  RT @cybercuichi: Miracles and Faith have a hol...
3999   246      0  RT @stoned_satan: https://t.co/y8SXRiJrP8 #cig...

[4000 rows x 3 columns]>

In [16]:
wandb.login()

wandb: Currently logged in as: liang_kdai. Use `wandb login --relogin` to force relogin


True

In [17]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [18]:
sentences = text_data.text.values
labels = text_data.label.values
test_sentences = test_data.text.values

In [19]:
input_ids = []
test_input_ids = []
attention_masks = []
test_attention_masks = []

for sent in sentences:
    encoded_dict = tokenizer.encode_plus(
                        sent,
                        add_special_tokens = True, # 添加 cls 与 sep
                        max_length = 130,
                        padding= "max_length",           # 使用 0-padding 统一长度
                        return_attention_mask = True,   # 构建注意力以及掩码
                        return_tensors = 'pt',     # 以 pytorch 的 tensor 类型返回
                   )
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

for sent in test_sentences:
    encoded_dict = tokenizer.encode_plus(
                        sent,
                        add_special_tokens = True,
                        max_length = 130,
                        padding= "max_length",
                        return_attention_mask = True,
                        return_tensors = 'pt',
                   )
    test_input_ids.append(encoded_dict['input_ids'])
    test_attention_masks.append(encoded_dict['attention_mask'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

test_input_ids = torch.cat(test_input_ids, dim=0)
test_attention_masks = torch.cat(test_attention_masks, dim=0)

print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])
print('Original: ', test_sentences[0])
print('Token IDs:', test_input_ids[0])

Original:  RT @AmitSwami77: The conspirators have an evil eye & are now set to physically attack Asaram Bapu Ji! #WeDemandSafety4Bapuji http://t.co/N8
Token IDs: tensor([  101, 19387,  1030, 26445,  3215,  4213,  4328,  2581,  2581,  1024,
         1996,  9530, 13102,  7895,  6591,  2031,  2019,  4763,  3239,  1004,
         2024,  2085,  2275,  2000,  8186,  2886, 17306,  6444,  8670, 14289,
        10147,   999,  1001, 21981, 16704,  5104, 10354, 27405,  2549,  3676,
        14289,  4478,  8299,  1024,  1013,  1013,  1056,  1012,  2522,  1013,
         1050,  2620,   102,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,  

In [20]:
dataset = TensorDataset(input_ids, attention_masks, labels)
test_dataset = TensorDataset(test_input_ids, test_attention_masks)

# 测试集九一分出测试集与验证集
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print(str(train_size) + " training samples")
print(str(val_size) + " validation samples")

3,600 training samples
  400 validation samples


In [22]:
def ret_dataloader():
    batch_size = wandb.config.batch_size
    print('batch_size = ', batch_size)
    train_dataloader = DataLoader(train_dataset,
                    sampler = RandomSampler(train_dataset), 
                    batch_size = batch_size)

    validation_dataloader = DataLoader(val_dataset, 
                      sampler = SequentialSampler(val_dataset),   
                      batch_size = batch_size)
    test_dataloader = DataLoader(test_dataset, 
                    sampler = SequentialSampler(test_dataset),   
                    batch_size = batch_size)
    return train_dataloader,validation_dataloader, test_dataloader

def ret_model():
    model = BertForSequenceClassification.from_pretrained(
        "bert-base-uncased", 
        num_labels = 3, 
        output_attentions = True, 
        output_hidden_states = True,
    )
    return model

def ret_optim(model):
    print('Learning_rate = ',wandb.config.learning_rate )
    optimizer = AdamW(model.parameters(),
                      lr = wandb.config.learning_rate, 
                      eps = wandb.config.epsilon
                    )
    return optimizer

def ret_scheduler(train_dataloader,optimizer):
    epochs = wandb.config.epochs
    total_steps = len(train_dataloader) * epochs
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0,           
num_training_steps = total_steps)
    return scheduler

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [24]:
wandb.init(project="multimodel_emotion_recognize",
      name="text",
      config={"epochs":10, "batch_size":16, "learning_rate":  2e-6, "epsilon": 1e-7})

wandb: Currently logged in as: liang_kdai. Use `wandb login --relogin` to force relogin


In [25]:
model = ret_model()
model.cuda()
train_dataloader,validation_dataloader, test_dataloader = ret_dataloader()
optimizer = ret_optim(model)
scheduler = ret_scheduler(train_dataloader,optimizer)
training_stats = []
total_t0 = time.time()
epochs = wandb.config.epochs

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

batch_size =  16
Learning_rate =  2e-06


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
if torch.cuda.is_available():       
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [27]:
training_stats = []
total_t0 = time.time()

for epoch_i in range(0, epochs): 
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()
    total_train_loss = 0
    total_train_accuracy = 0

    model.train()

    for step, batch in enumerate(train_dataloader):
        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        model.zero_grad()        
        output = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)

        wandb.log({'train_batch_loss':output.loss.item()})
        total_train_loss += output.loss.item()
        output.loss.backward()
        quit()
        logits = output.logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        total_train_accuracy += flat_accuracy(logits, label_ids)
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_train_loss / len(train_dataloader)  
    avg_train_accuracy = total_train_accuracy / len(train_dataloader) 
    print("Train  Accuracy: {0:.2f}".format(avg_train_accuracy))

    wandb.log({'avg_train_loss':avg_train_loss,'avg_train_acc':avg_train_accuracy})
    training_time = format_time(time.time() - t0)

    print("\n  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}\n".format(training_time))
    print("Running Validation...")

    t0 = time.time()

    model.eval()
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    for batch in validation_dataloader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        with torch.no_grad():        
            output = model(b_input_ids, 
                  token_type_ids=None, 
                  attention_mask=b_input_mask,
                  labels=b_labels)
            
        total_eval_loss += output.loss.item()
        # 数值计算的时候需要将 cuda 的数据转回 cpu
        logits = output.logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))
    wandb.log({'val_accuracy':avg_val_accuracy,'avg_val_loss':avg_val_loss})

    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("\nTraining complete!")
print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))
torch.save(model, "save_model.pth")


======== Epoch 1 / 10 ========
Training...
  Batch    40  of    225.    Elapsed: 0:00:14.
  Batch    80  of    225.    Elapsed: 0:00:28.
  Batch   120  of    225.    Elapsed: 0:00:42.
  Batch   160  of    225.    Elapsed: 0:00:56.
  Batch   200  of    225.    Elapsed: 0:01:10.
Train  Accuracy: 0.58

  Average training loss: 0.92
  Training epcoh took: 0:01:20

Running Validation...
  Accuracy: 0.64
  Validation Loss: 0.84
  Validation took: 0:00:03

======== Epoch 2 / 10 ========
Training...
  Batch    40  of    225.    Elapsed: 0:00:15.
  Batch    80  of    225.    Elapsed: 0:00:29.
  Batch   120  of    225.    Elapsed: 0:00:44.
  Batch   160  of    225.    Elapsed: 0:00:59.
  Batch   200  of    225.    Elapsed: 0:01:13.
Train  Accuracy: 0.63

  Average training loss: 0.84
  Training epcoh took: 0:01:23

Running Validation...
  Accuracy: 0.69
  Validation Loss: 0.75
  Validation took: 0:00:03

======== Epoch 3 / 10 ========
Training...
  Batch    40  of    225.    Elapsed: 0:00:15.
 

In [42]:
# 使用模型对测试集的 txt 文本进行情感预测
model1 = torch.load("save_model.pth")
preds = []
model1.eval()

for batch in test_dataloader:
  b_input_ids = batch[0].to(device)
  b_input_mask = batch[1].to(device)
  
  with torch.no_grad():        
      output = model1(b_input_ids, 
              token_type_ids=None, 
              attention_mask=b_input_mask)
      output_ = output[0].cpu().numpy() # 同样需要先转成 cpu 才能继续计算
      for i in output_:
        preds.append(i.argmax())

In [43]:
print(preds)

[2, 0, 2, 0, 2, 2, 2, 2, 2, 0, 2, 0, 2, 2, 2, 2, 2, 2, 1, 0, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 0, 0, 2, 0, 2, 2, 2, 2, 0, 2, 2, 0, 0, 1, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 0, 0, 2, 2, 0, 0, 2, 0, 0, 2, 0, 2, 2, 0, 2, 2, 2, 2, 0, 2, 2, 2, 2, 0, 2, 2, 2, 2, 0, 2, 2, 0, 2, 2, 0, 2, 0, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 0, 0, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 2, 0, 0, 0, 2, 2, 0, 0, 0, 0, 2, 0, 2, 0, 0, 2, 2, 0, 0, 2, 2, 2, 2, 2, 0, 2, 0, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 0, 0, 0, 2, 0, 2, 2, 0, 2, 0, 0, 0, 2, 0, 2, 2, 2, 0, 2, 2, 0, 0, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 0, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 0, 0, 0, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 0, 0, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 0, 0, 0, 0, 2, 0, 2, 2, 2, 2, 2, 2, 0, 2, 0, 2, 2, 0, 2, 2, 2, 2, 0, 2, 2, 2, 0, 0, 2, 2, 0, 2, 2, 0, 1, 2, 0, 2, 2, 0, 2, 1, 2, 2, 0, 2, 0, 2, 2, 2, 2, 2, 2, 0, 0, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 0, 0, 2, 2, 2, 2, 0, 2, 0, 0, 

In [46]:
def label2tag(label):
  label = int(label)
  if label == 0:  return "negative"
  elif label == 1:  return "neutral"
  else: return "positive"

In [47]:
with open('test_with_label.txt', 'w') as f:  # 输出预测结果
  f.write("guid,tag\n")
  for i in range(len(test_list)):
    f.write(str(test_list[i])+","+label2tag(preds[i])+"\n")